# **Análise Semântica e Preditiva de Padrões Criminais**
### **Modelagem Estocástica utilizando NLP (Transformers), Teoria dos Grafos e Cadeias de Markov**


## Introdução
**Contexto do Projeto**: A análise criminal tradicional frequentemente se limita a dados geospaciais e numéricos. No entanto, uma grande quantidade de informações reside nos dados não estruturados: as descrições textuais dos Boletins de Ocorrência.

**O Problema**: Como identificar conexões ocultas entre diferentes municípios baseando-se não na proximidade física, mas na similaridade do modus operandi dos crimes? E, uma vez mapeada essa rede, como prever para onde a estrutura do sistema direciona a probabilidade de novas ocorrências?

**Abordagem**: Este projeto propõe uma arquitetura híbrida:

1. **NLP (Processamento de Linguagem Natural)**: Utilização de modelos Transformer (SBERT) para converter descrições textuais em vetores semânticos de alta dimensão.

2. **Network Science**: Construção de um grafo de similaridade onde as conexões representam a afinidade semântica entre crimes de diferentes cidades.

3. **Modelagem Probabilística**: Aplicação de Cadeias de Markov para estimar a distribuição estacionária, identificando "hubs" de criminalidade com maior probabilidade teórica de reincidência de padrões específicos.

## Metodologia Técnica

O pipeline de dados foi estruturado em três etapas principais:

1. **Vetorização Semântica (Embeddings)**: Utilizei o modelo pré-treinado paraphrase-multilingual-mpnet-base-v2 da biblioteca sentence-transformers. Este modelo mapeia as sentenças para um espaço vetorial denso de 768 dimensões, capturando nuances semânticas que abordagens baseadas em frequência (como TF-IDF) perderiam.

2. **Modelagem de Grafo e Filtragem**: Calculei a matriz de similaridade do cosseno entre todas as descrições. Para garantir a relevância topológica do grafo, apliquei um threshold (limiar de corte) rigoroso. Apenas arestas com similaridade semântica muito alta (> 0.85) foram mantidas, criando uma rede que conecta municípios apenas quando os crimes descritos são idênticos em natureza.

3. **Cadeias de Markov**: Transformei a matriz de adjacência do grafo em uma Matriz de Transição Estocástica. Assumindo que a propagação de padrões criminais na rede segue um processo de Random Walk (Caminhada Aleatória), calculei o vetor de probabilidade estacionária ($\pi$). Isso permite prever quais nós (cidades) atuam como atratores centrais na rede de padrões criminais.

## Desenvolvimento: Modelagem do Grafo

### **Importações**

In [ ]:
!pip install -U sentence-transformers

In [ ]:
!pip install pyvis

In [ ]:
!pip install rapidfuzz

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
import seaborn as sns
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from rapidfuzz import process
from wordcloud import WordCloud, STOPWORDS

### **Estratégia de Modelagem da Rede**

Para transformar a similaridade semântica em uma estrutura de rede navegável, adotei uma abordagem de filtragem em duas etapas:

1. **Redução de Dimensionalidade**: Inicialmente, calculei a similaridade de cosseno entre os embeddings, retendo conexões com score $\ge 0.7$. Deste conjunto, extraí um subgrafo (G_top) focado nos 7.000 nós de maior grau (hubs), garantindo que a análise se concentrasse nas regiões de maior densidade informacional e eliminando ruídos periféricos.

2. **Refinamento Semântico**: Para a visualização e detecção de comunidades, apliquei threshold >= 0.85. Isso isolou as conexões "fortes", onde as descrições dos crimes são virtualmente idênticas, permitindo a identificação clara de padrões de modus operandi compartilhados entre cidades distintas.

In [ ]:
!gdown 1168-rcUkZWtKk2CfievZdVrQNpPV2m_8
df = pd.read_excel('assaltos.xlsx')
df

### Mitigação de Viés de Boilerplate

Em registros policiais, é comum o uso de frases padrão curtas (ex: "Roubo a transeunte" ou "Furto de celular").

Ao manter esses registros, o modelo SBERT atribuirá similaridade semântica de 100% entre cidades que apenas usam o mesmo "jargão burocrático", criando falsos hubs de criminalidade baseados em preguiça de preenchimento e não em modus operandi real.

**A Solução**: Apliquei um filtro de complexidade narrativa, descartando ocorrências com menos de 5 palavras. Isso força o modelo a encontrar conexões baseadas em detalhes específicos (tipo de arma, veículo utilizado, dinâmica da abordagem), aumentando a qualidade preditiva da rede.

In [ ]:
# Filtro de qualidade da informação
# Objetivo: Remover descrições genéricas para evitar
# conexões artificiais baseadas apenas na falta de detalhes

# Limite: Manter apenas descrições com 5 ou mais palavras
min_palavras = 5

print(f"Tamanho original do dataset: {len(df)} ocorrências")

# Aplicação do Filtro
df_clean = df[df['descricao'].str.split().str.len() >= min_palavras].copy()

# Impacto
removidos = len(df) - len(df_clean)
print(f"Registros removidos (Curto/Genérico): {removidos}")
print(f"Tamanho final para análise: {len(df_clean)} ocorrências ricas")

df = df_clean
print("\nExemplos de descrições mantidas:")
print(df['descricao'].head(3).values)

Tamanho original do dataset: 15636 ocorrências
Registros removidos (Curto/Genérico): 2
Tamanho final para análise: 15634 ocorrências ricas

Exemplos de descrições mantidas:
["Moradores do Ipiranga, na Zona Sul, sofrem com três dias consecutivos de assaltos no bairro: 'Minuto mais longo da minha vida' - G1"
 'Câmera registra assalto na zona leste de SP: “Mata, mata“, diz ladrão - CNN Brasil'
 'Cidade de São Paulo tem queda de roubos e aumento de furtos em janeiro - Poder360']


In [ ]:
# Embeddings de descrições

# Amostragem
df_sample = df.sample(n=15600, random_state=42).reset_index(drop=True)

# Embeddings
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")
sentences = df_sample["descricao"].tolist()
embeddings = model.encode(sentences, show_progress_bar=True)

In [ ]:
# Matriz de similaridade

# Calcular matriz de similaridade
similarity_matrix = cosine_similarity(embeddings)

# Construção de arestas baseada em threshold semântico
def construir_grafo(cidades, sim_matrix, threshold=0.7):
    G = nx.Graph()
    G.add_nodes_from(cidades)
    n = len(cidades)
    for i in range(n):
        for j in range(i + 1, n):
            sim = sim_matrix[i][j]
            if sim >= threshold:
                G.add_edge(cidades[i], cidades[j], weight=sim)
    return G

# Construir o grafo
cidades = df_sample["municipio"].tolist()
G = construir_grafo(cidades, similarity_matrix, threshold=0.7)

# Filtragem dos principais hubs para otimização de processamento
top_x = sorted(G.degree, key=lambda x: x[1], reverse=True)[:10000]
top_cities = [cidade for cidade, grau in top_x]

# Criar subgrafo com as cidades mais conectadas
G_top = G.subgraph(top_cities).copy()



In [ ]:
# Verificar conectividade para prosseguir a análise ou mudar a abordagem
print("Grafo é conexo?", nx.is_connected(G))
print("Componentes conectados:", nx.number_connected_components(G))

Grafo é conexo? True
Componentes conectados: 1


### **Análise da Distribuição de Similaridades**

A distribuição dos scores de similaridade segue uma curva aproximadamente Normal (Gaussiana). Isso revela que a grande maioria das conexões entre municípios representa apenas ruído semântico (uso de vocabulário jurídico padrão e palavras comuns).

In [ ]:
n_amostras = 5000
if 'embeddings' in locals() and len(embeddings) > n_amostras:
    indices = np.random.choice(len(embeddings), n_amostras, replace=False)
    embeddings_sample = embeddings[indices]
else:
    if 'embeddings' in locals():
        embeddings_sample = embeddings
    else:
        print("Usando dados simulados para teste visual...")
        embeddings_sample = np.random.rand(100, 768)

sim_matrix_sample = cosine_similarity(embeddings_sample)
mask = np.triu_indices(sim_matrix_sample.shape[0], k=1)
sim_values = sim_matrix_sample[mask]
threshold_corte = 0.85

plt.figure(figsize=(10, 6))
sns.histplot(sim_values, bins=50, color='#4c72b0', edgecolor=None, kde=True, alpha=0.6, label='Distribuição Geral')

plt.axvline(threshold_corte, color='#c44e52', linestyle='--', linewidth=2, label=f'Threshold ({threshold_corte})')
media_sim = np.mean(sim_values)
y_max = plt.gca().get_ylim()[1]
plt.text(media_sim, y_max * 0.5, 'Ruído Semântico\n(Descrições Genéricas)',
         horizontalalignment='center', color='#555555', fontsize=10, style='italic')

plt.text(threshold_corte + 0.02, y_max * 0.2, 'Clusters de\nAlta Similaridade',
         color='#c44e52', fontsize=10, fontweight='bold')

plt.title("Distribuição de Similaridade Semântica e Ponto de Corte", fontsize=14, pad=20)
plt.xlabel("Grau de Similaridade (Cosseno)", fontsize=11)
plt.ylabel("Densidade / Frequência", fontsize=11)

plt.yticks([])
plt.grid(axis='y', alpha=0.2, linestyle='--')
plt.legend(frameon=False)

ax = plt.gca()
ax.spines['top'].set_visible(True)
ax.spines['right'].set_visible(True)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(True)

plt.tight_layout()
plt.savefig("distribuicao_similaridade_semantica.png", format="png", dpi=300, bbox_inches='tight')
plt.show()

### **Interpretação do Grafo**

* **Visualização da Rede Semântica**: Cada nó representa um município. As arestas conectam cidades com descrições de crimes semanticamente similares (Cosine Similarity > 0.85). O tamanho e a cor dos nós são proporcionais ao Grau de Conexão, destacando os "hubs" de padrões criminais.


In [ ]:
# Filtra arestas do subgrafo com peso acima do threshold
threshold_arestas = 0.85
arestas_filtradas = [
    (u, v, d) for u, v, d in G_top.edges(data=True) if d['weight'] >= threshold_arestas
]

# Cria um novo grafo contendo apenas os nós originais e as arestas fortes, que estão acima do limiar definido
G_filtrado = nx.Graph()
G_filtrado.add_nodes_from(G_top.nodes(data=True))
G_filtrado.add_edges_from(arestas_filtradas)
plt.figure(figsize=(20, 15))

pos = nx.spring_layout(G_filtrado, seed=42, k=0.6)
graus_top = dict(G_filtrado.degree())
node_colors = [graus_top[node] for node in G_filtrado.nodes()]
node_sizes = [300 + graus_top[node]*50 for node in G_filtrado.nodes()]

edge_weights = [G_filtrado[u][v]['weight'] for u, v in G_filtrado.edges()]

nx.draw_networkx_nodes(
    G_filtrado, pos,
    node_size=node_sizes,
    node_color=node_colors,
    cmap=plt.cm.plasma,
    alpha=0.9
)

nx.draw_networkx_edges(
    G_filtrado, pos,
    width=[w*0.7 for w in edge_weights],
    edge_color='gray',
    alpha=0.5
)

top_10_cities = sorted(graus_top.items(), key=lambda x: x[1], reverse=True)[:10]
labels = {cidade: cidade for cidade, _ in top_10_cities}

nx.draw_networkx_labels(G_filtrado, pos, labels, font_size=9)

legend_handles = [
    mpatches.Patch(color=plt.cm.plasma(0.2), label="Baixa conectividade"),
    mpatches.Patch(color=plt.cm.plasma(0.5), label="Conectividade média"),
    mpatches.Patch(color=plt.cm.plasma(0.9), label="Alta conectividade")
]

plt.legend(handles=legend_handles, title="Grau de Conexão", loc='upper right')

plt.title("Grafo de Similaridade Semântica entre cidades com mais conexões - Assaltos", fontsize=14)
plt.axis("off")
plt.savefig("grafo.png", format="png", dpi=300, bbox_inches='tight')
plt.show()


> **Visualização da Topologia da Rede:**
> * **Nós (Pontos):** Municípios.
> * **Arestas (Linhas):** Conexões semânticas fortes (Similaridade > 0.85).
> * **Cor/Tamanho:** Proporcional ao Grau (Degree Centrality).
>
> *Nota-se a formação de "clusters" densos (roxo/rosa), indicando grupos de cidades que compartilham descrições de crimes padronizadas, enquanto nós isolados representam ocorrências atípicas.*

### **Análise Topológica e Padrões de Ocorrência**

A estrutura do grafo revelou três comportamentos distintos na base de dados:

1. **Hubs de Alta Centralidade**: Cidades com alto grau de conexão (nós maiores/mais claros) indicam a presença de descrições genéricas ou modus operandi padronizados que se repetem em larga escala nacionalmente.

2. **Clusters Regionais**: Agrupamentos densos sugerem que certas regiões compartilham vocabulário específico ou tipos de crimes muito característicos (ex. gírias locais ou crimes específicos de fronteira).

3. **Outliers**: Cidades desconectadas apontam para ocorrências com descrições únicas, que podem derivar de crimes atípicos ou anomalias na qualidade do preenchimento dos dados (ruído).


### **Cidades com maior Grau de Conexão**

**Análise de Centralidade (Hubs Semânticos)**

Ao calcular o grau dos vértices, identifiquei as cidades que atuam como "hubs" na rede. Uma alta centralidade aqui não implica necessariamente em maior volume de crimes, mas sim que os crimes ocorridos nestas cidades possuem descrições que utilizam padrões narrativos e palavras chave que se conectam com centenas de outros registros na base. Estas cidades representam o "padrão médio" da criminalidade nacional na amostra.

In [ ]:
graus = dict(G.degree())
top_cities = sorted(graus.items(), key=lambda x: x[1], reverse=True)
print("Top 5 cidades com maior grau de conexão:")
for cidade, grau in top_cities[:5]:
    print(f"{cidade}: {grau}")


Top 5 cidades com maior grau de conexão:
São Carlos, SP, BRAZIL: 311
João Pessoa, PB, BRAZIL: 311
São José do Rio Preto, SP, BRAZIL: 308
São Luís, MA, BRAZIL: 306
São Vicente, SP, BRAZIL: 304


### **Municípios com maior similaridade**

**Análise de Pares Críticos (Similaridade Máxima)**

Investigando as arestas com maior peso (Top Pairs), observei conexões com similaridade de cosseno próxima ou igual a 1.0. Isso indica ocorrências com descrições textuais virtualmente idênticas entre municípios distintos, sugerindo dois cenários:

* **Padronização Administrativa**: O uso de templates ou textos pré-formatados no preenchimento dos Boletins de Ocorrência por diferentes delegacias.

* **Replicabilidade Exata**: Crimes com modus operandi extremamente específico e estruturado (ex: ataques a bancos ou roubo de carga) que geram descrições técnicas idênticas.

In [ ]:
df_top = df_sample[df_sample['municipio'].isin(top_cities)].copy()
top_n = 5

# Ordena as arestas do grafo filtrado pela força da conexão, do maior para o menor
arestas_ordenadas = sorted(
    G_filtrado.edges(data=True),
    key=lambda x: x[2]['weight'],
    reverse=True
)

# Função para buscar uma descrição aproximada com base no nome do município
def buscar_descricao_fuzzy(nome, lista_municipios, descricoes):
    match, score, idx = process.extractOne(nome, lista_municipios)

    # Se a similaridade for suficientemente alta, retorna a descrição correspondente
    if score >= 80:
        return descricoes[idx]
    else:
        return "(descrição não encontrada)"
lista_municipios = df_sample['municipio'].tolist()
descricoes = df_sample['descricao'].tolist()

for u, v, data in arestas_ordenadas[:top_n]:
    sim = data['weight']

    # Busca as descrições das duas cidades envolvidas na conexão
    desc_u = buscar_descricao_fuzzy(u, lista_municipios, descricoes)
    desc_v = buscar_descricao_fuzzy(v, lista_municipios, descricoes)

    print(f"\n Similaridade: {sim:.4f} entre '{u}' e '{v}'")
    print(f" Descrição de {u}: {desc_u}")
    print(f" Descrição de {v}: {desc_v}")



 Similaridade: 1.0000 entre 'Conselheiro Lafaiete, MG, BRAZIL' e 'Barbacena, MG, BRAZIL'
 Descrição de Conselheiro Lafaiete, MG, BRAZIL: Posto de combustíveis é alvo de tentativa de roubo em Conselheiro Lafaiete - Vertentes das Gerais
 Descrição de Barbacena, MG, BRAZIL: Disputa política pode ter motivado morte de homem em Cipotânea - Lafaiete Agora

 Similaridade: 1.0000 entre 'Betim, MG, BRAZIL' e 'Porto Alegre, RS, BRAZIL'
 Descrição de Betim, MG, BRAZIL: Homem é morto a tiros em Betim - G1
 Descrição de Porto Alegre, RS, BRAZIL: Operação Brush desencadeia megaoperação contra o crime organizado no RS - Terra

 Similaridade: 1.0000 entre 'São Luís, MA, BRAZIL' e 'Balsas, MA, BRAZIL'
 Descrição de São Luís, MA, BRAZIL: VÍDEO: Motorista de ônibus é morto em assalto nesta segunda (22) em São Luís - Diário do Transporte
 Descrição de Balsas, MA, BRAZIL: Jovens são mortos a tiros em Balsas; crime teria sido motivado por briga de organizações criminosas - G1

 Similaridade: 1.0000 entre '

## Desenvolvimento: Modelagem Preditiva - Dinâmica Estocástica e Cadeias de Markov

Uma vez mapeada a estrutura estática da rede de similaridade, o próximo passo é modelar a dinâmica do sistema. A hipótese central desta etapa é que padrões criminais não são eventos isolados, mas tendem a se replicar através da rede de conexões semânticas.
Utilizando Cadeias de Markov, transformei o grafo em um sistema de estados onde:

1. **Estados**: São os municípios.
2. **Transições**: A probabilidade de um padrão criminal "migrar" de uma cidade para outra é proporcional à similaridade das descrições dos crimes.

O objetivo é calcular a Distribuição Estacionária ($\pi$), que revela a probabilidade a longo prazo de ocorrência de crimes em cada nó, independentemente do estado inicial. Cidades com alta probabilidade estacionária representam os "atratores" do sistema.

### **Modelagem Estocástica: Construção da Matriz de Transição**

Para aplicar as Cadeias de Markov, é necessário converter a representação topológica do grafo (Matriz de Adjacência) em uma representação probabilística (Matriz de Transição Estocástica).

**Lógica da Conversão**:

**1. Mapeamento**: Cada peso de aresta (similaridade do cosseno) foi mapeado para uma célula da matriz.

**2. Normalização**: Normalizei os valores linha a linha para que o somatório de cada linha seja igual a $1$.

**Interpretação**: Nesta etapa, a similaridade semântica deixa de ser uma métrica de comparação e passa a representar a probabilidade de transição. Ou seja, se um padrão criminal está na Cidade A, a probabilidade dele "migrar" para a Cidade B é proporcional à similaridade das descrições de crimes entre elas.

In [ ]:
# Construção da matriz de adjacência usando G_top e as cidades do subgrafo
cidades_ordenadas = list(G_top.nodes())
n = len(cidades_ordenadas)

adj_matrix = np.zeros((n, n))

for i, u in enumerate(cidades_ordenadas):
    for j, v in enumerate(cidades_ordenadas):
        if G_top.has_edge(u, v):
            adj_matrix[i, j] = G_top[u][v]['weight']

# Normalização linha a linha para criar matriz de transição
transition_matrix = np.zeros_like(adj_matrix)
row_sums = adj_matrix.sum(axis=1)
eps = 1e-12

for i in range(n):
    if row_sums[i] < eps:
        # Se a linha tem soma quase zero, assume distribuição uniforme
        transition_matrix[i] = np.ones(n) / n
    else:
        # Caso contrário, normaliza a linha para que a soma seja 1
        transition_matrix[i] = adj_matrix[i] / row_sums[i]



### **Cidades com Maior Probabilidade Estacionária**

Com base na convergência da Cadeia de Markov, isolei os 10 municípios com maior Probabilidade Estacionária ($\pi$).

In [ ]:
# Distribuição Estacionária
pi = np.ones(n) / n
for _ in range(100):
    pi_next = pi @ transition_matrix
    if np.allclose(pi, pi_next, atol=1e-8):
        break
    pi = pi_next


# Probabilidades finais
df_markov = pd.DataFrame({
    'municipio': cidades_ordenadas,
    'prob_assalto_futuro': pi
}).sort_values(by='prob_assalto_futuro', ascending=False)

print("Top 10 cidades com maior probabilidade futura de ocorrência de assaltos - Cadeia de Markov")
print(df_markov.head(10))

Top 10 cidades com maior probabilidade futura de ocorrência de assaltos - Cadeia de Markov
                             municipio  prob_assalto_futuro
54              São Carlos, SP, BRAZIL             0.004334
58             João Pessoa, PB, BRAZIL             0.004317
43   São José do Rio Preto, SP, BRAZIL             0.004286
12                São Luís, MA, BRAZIL             0.004262
157            São Vicente, SP, BRAZIL             0.004259
34          Ribeirão Preto, SP, BRAZIL             0.004186
78        Feira de Santana, BA, BRAZIL             0.004175
131               Santarém, PA, BRAZIL             0.004156
119                  Serra, ES, BRAZIL             0.004138
19              Vila Velha, ES, BRAZIL             0.004134


### **Cidades com Maior Probabilidade Estacionária**

O cálculo do vetor estacionário ($\pi$) revela os "atratores" do sistema. As cidades abaixo apresentam a maior probabilidade teórica de convergir padrões criminais, devido à sua posição estratégica na rede semântica

In [ ]:
top_n = 10
df_top = df_markov.sort_values(by='prob_assalto_futuro', ascending=False).head(top_n).copy()

df_top['municipio_limpo'] = df_top['municipio'].str.replace(', BRAZIL', '', regex=False)
df_top_sorted = df_top.sort_values(by='prob_assalto_futuro', ascending=False)

norm = mcolors.Normalize(
    vmin=df_top_sorted['prob_assalto_futuro'].min(),
    vmax=df_top_sorted['prob_assalto_futuro'].max()
)
cmap = plt.cm.Reds
colors = [cmap(norm(p)) for p in df_top_sorted['prob_assalto_futuro']]
plt.figure(figsize=(10, 6))

ax = sns.barplot(
    data=df_top_sorted,
    x='prob_assalto_futuro',
    y='municipio_limpo',
    hue='municipio_limpo',
    palette=colors,
    dodge=False
)

for container in ax.containers:
    ax.bar_label(container, fmt='%.5f', padding=5, fontsize=10, color='black')

plt.title("10 Cidades com Maior Probabilidade Estacionária de Crimes", fontsize=14, pad=20)
plt.ylabel("")
plt.xlabel("Probabilidade Estimada (Markov)", fontsize=10)

if ax.legend_:
    ax.legend_.remove()

plt.xticks([])
sns.despine(left=True, bottom=True)

plt.tight_layout()
plt.savefig("top_10_cidades_probabilidade_crimes.png", format="png", dpi=300, bbox_inches='tight')
plt.show()

**Interpretação do Modelo**: Diferente de um ranking baseado apenas em volume bruto de ocorrências, este resultado reflete a Força Estrutural na rede semântica.
* As cidades listadas acima funcionam como "Atratores de Padrões": devido à sua alta conectividade com descrições de crimes de todo o país, o modelo prevê que, estatisticamente, a tendência é que novos padrões criminais convirjam para características similares às observadas nestes locais.
* A graduação de cor (vermelho mais escuro) indica maior intensidade na probabilidade estimada, destacando as zonas críticas do sistema.

### **Validação do Modelo**

Para garantir a robustez dos resultados algébricos, realizei uma validação cruzada através de simulação.

**Metodologia**: Executei uma Random Walk com 100.000 passos na matriz de transição. O objetivo é verificar se a frequência empírica de visitas converge para a probabilidade teórica calculada ($\pi$).

In [ ]:
# Simulação de Monte Carlo
num_passos = 100000
visitas = np.zeros(n)
estado_atual = np.random.choice(n)

for _ in range(num_passos):
    visitas[estado_atual] += 1
    probs = transition_matrix[estado_atual]
    estado_atual = np.random.choice(n, p=probs)

frequencia_simulada = visitas / visitas.sum()

df_resultado = pd.DataFrame({
    "municipio": cidades_ordenadas,
    "Teórica (Estacionária)": pi,
    "Empírica (Simulada)": frequencia_simulada
}).sort_values(by="Teórica (Estacionária)", ascending=False)

top_n = 5
df_top = df_resultado.head(top_n).copy()
df_top['municipio_limpo'] = df_top['municipio'].str.replace(', BRAZIL', '', regex=False)

y = np.arange(top_n)
height = 0.35

fig, ax = plt.subplots(figsize=(12, 8))

bars1 = ax.barh(y - height/2, df_top['Teórica (Estacionária)'], height,
                label='Teórica (Estacionária)', color='#1f77b4', alpha=0.8)
bars2 = ax.barh(y + height/2, df_top['Empírica (Simulada)'], height,
                label='Empírica (Simulada)', color='#ff7f0e', alpha=0.8)

ax.bar_label(bars1, fmt='%.5f', padding=3, fontsize=9, color='black')
ax.bar_label(bars2, fmt='%.5f', padding=3, fontsize=9, color='black')

ax.set_ylabel('')
ax.set_xlabel('Probabilidade / Frequência', fontsize=10)
ax.set_title('Validação Cruzada: Distribuição Teórica vs. Simulação de Monte Carlo', fontsize=14, pad=20)

ax.set_yticks(y)
ax.set_yticklabels(df_top['municipio_limpo'], fontsize=11)
ax.invert_yaxis()

ax.set_xticks([])
sns.despine(left=True, bottom=True)
ax.legend(loc='lower center', bbox_to_anchor=(0.5, 0.96), ncol=2, frameon=False, fontsize=10)

plt.tight_layout()
plt.savefig("validacao_cruzada_simulacao_monte_carlo.png", format="png", dpi=300, bbox_inches='tight')
plt.show()

### **Interpretação**

* A coluna estacionária representa o que o modelo de Markov preveria após muitos passos (como se o tempo fosse infinito).

* A coluna simulada representa o que de fato foi observado ao simular a cadeia várias vezes (pode ter pequenas variações).

* Ambas servem para classificar os municípios segundo o risco futuro de assaltos.



### **Análise de Convergência**

O gráfico comparativo demonstra uma sobreposição quase perfeita entre a Distribuição Estacionária (Teórica) e a Frequência Simulada.

**Conclusão**: A convergência valida a integridade da Matriz de Transição construída. O modelo é matematicamente estável e as probabilidades calculadas representam fielmente a dinâmica topológica do grafo de crimes.

### **Validação dos resultados da cadeia de Markov**


### **Análise dos Resultados**

**Topologia da Rede**: A visualização do grafo revelou a existência de clusters densamente conectados. Cidades como João Pessoa (PB) e São Carlos (SP) emergiram como "Hubs Semânticos". Isso indica que os tipos de crimes ocorridos nestas localidades possuem características descritivas que se repetem em diversos outros municípios, tornando-os centrais na rede de similaridade.

**Previsão Estocástica**: O modelo de Markov identificou que a probabilidade de ocorrência futura não é uniforme. O vetor estacionário aponta que as cidades mais conectadas semanticamente têm uma probabilidade significativamente maior de absorver padrões criminais da rede.

**Interpretação de Negócio**: A alta similaridade (muitas vezes chegando a 1.0) entre descrições de cidades distantes sugere padrões de preenchimento de B.O. padronizados ou ondas de crimes com características muito específicas (ex: "roubo de carga" ou "ataque a carro-forte") que se replicam nacionalmente.

* A topologia resultante revela clusters baseados na natureza do evento, e não na geografia.

* **Alta Centralidade (Hubs)**: Cidades com descrições genéricas ou padronizadas que se conectam a múltiplos locais.

* **Clusters Isolados**: Indicam ocorrências com descrições muito específicas ou anomalias nos dados.

**1. Diagnóstico de Integridade Matemática (Sanity Check)**

Antes de concluir a análise, realizei verificações de integridade para garantir que as propriedades fundamentais das Cadeias de Markov foram respeitadas:

* **Estocasticidade**: A soma das probabilidades de transição de cada estado deve ser igual a $1.0$.

* **Normalização**: O vetor estacionário ($\pi$) deve somar $1.0$ e não conter valores negativos.

In [ ]:
# Verificação de Integridade do Modelo

# Verificar Estocasticidade da Matriz de Transição
soma_linhas = transition_matrix.sum(axis=1)
assert np.allclose(soma_linhas, 1.0, atol=1e-8), "Erro: A matriz não é estocástica!"

# Verificar Propriedades do Vetor Estacionário (pi)
assert np.isclose(pi.sum(), 1.0), "Erro: A soma de Pi deve ser 1.0"
assert (pi >= -1e-12).all(), "Erro: Existem probabilidades negativas"

# Output limpo de sucesso
print("Diagnóstico Concluído com SUCESSO:")
print(f"   - Matriz Estocástica: OK (Min: {soma_linhas.min():.4f}, Max: {soma_linhas.max():.4f})")
print(f"   - Vetor Estacionário: OK (Soma: {pi.sum():.4f})")

Diagnóstico Concluído com SUCESSO:
   - Matriz Estocástica: OK (Min: 1.0000, Max: 1.0000)
   - Vetor Estacionário: OK (Soma: 1.0000)


**2. Validação das Propriedades Probabilísticas ($\pi$)**

Para confirmar a consistência matemática do modelo, verifiquei se o vetor estacionário calculado atende aos axiomas fundamentais de probabilidade:

* **Unitariedade**: A soma de todas as probabilidades deve ser igual a $1.0$.
* **Não-Negatividade**: Nenhuma cidade pode ter probabilidade de ocorrência negativa.
* **Estabilidade Numérica**: Ausência de valores nulos (NaN) ou infinitos (Inf) decorrentes de divisões por zero.

In [ ]:
# Validação de Integridade do Vetor Estacionário

# Verifica se a soma é 1.0
assert np.isclose(pi.sum(), 1.0, atol=1e-8), f"Erro Crítico: A soma das probabilidades é {pi.sum()}"

# Verifica se existem valores negativos
assert np.all(pi >= -1e-12), "Erro Crítico: Existem probabilidades negativas no vetor."

# Verifica estabilidade numérica
assert not np.any(np.isnan(pi)), "Erro Crítico: O vetor contém valores NaN."

# Output limpo apenas se passar nos testes
print("Vetor Estacionário validado com SUCESSO.")
print(f"   - Soma Total: {pi.sum():.10f}")
print(f"   - Range de Probabilidades: [{pi.min():.6f}, {pi.max():.6f}]")

Vetor Estacionário validado com SUCESSO.
   - Soma Total: 1.0000000000
   - Range de Probabilidades: [0.000475, 0.004334]


**3. Teste de Convergência Numérica (Método da Potência)**

Para garantir que o vetor estacionário encontrado é a solução única e estável do sistema, monitorei a taxa de convergência do algoritmo. Utilizei a norma $L_1$ da diferença entre vetores consecutivos ($\|\pi_{t+1} - \pi_t\|_1$) como critério de parada.

* **Critério de Sucesso**: O erro residual ($\Delta$) deve ser inferior a $10^{-8}$.

In [ ]:
# Verificação de Convergência do Algoritmo
pi_test = np.ones(n) / n  # Inicialização uniforme
tolerancia = 1e-8
max_iter = 100
convergiu = False

for i in range(max_iter):
    pi_new = pi_test @ transition_matrix

    # Cálculo da Norma L1
    diff = np.linalg.norm(pi_new - pi_test, ord=1)

    if diff < tolerancia:
        convergiu = True
        print(f"Convergência Atingida.")
        print(f"   - Iterações necessárias: {i+1}")
        print(f"   - Erro Residual (Delta): {diff:.10e}") #
        break

    pi_test = pi_new

if not convergiu:
    print(f"ATENÇÃO: O algoritmo não convergiu após {max_iter} passos. Delta final: {diff}")

Convergência Atingida.
   - Iterações necessárias: 9
   - Erro Residual (Delta): 1.2505485081e-09


**4. Análise Qualitativa: Inspeção Semântica dos Atratores**

Após identificar os municípios com maior probabilidade estacionária, realizei uma inspeção manual das descrições textuais para validar a coerência semântica do agrupamento.

* **Hipótese de Validação**: Espera-se que os "hubs" identificados pelo modelo contenham descrições com alta densidade de termos críticos (ex: tiro, armado, refém, óbito) ou estruturas de relato padronizadas, o que explicaria sua alta conectividade vetorial com o restante da base.


In [ ]:
# Inspeção das descrições dos 5 principais "Hubs" de criminalidade
print(f"{'MUNICÍPIO':<30} | {'AMOSTRA DA DESCRIÇÃO'}")
print("-" * 100)

for i in range(5):
    cidade = df_markov.iloc[i]["municipio"]
    descricao = df_sample[df_sample["municipio"] == cidade]["descricao"].values[0]
    desc_curta = (descricao[:95] + '...') if len(descricao) > 95 else descricao
    print(f"{cidade:<30} | {desc_curta}")

MUNICÍPIO                      | AMOSTRA DA DESCRIÇÃO
----------------------------------------------------------------------------------------------------
São Carlos, SP, BRAZIL         | Assaltantes são presos após cerco policial em Ibaté - São Carlos Agora
João Pessoa, PB, BRAZIL        | Homens armados rendem mulheres e fazem 'troca de carros' no Pedro Gondim, em João Pessoa - G1
São José do Rio Preto, SP, BRAZIL | Dois homens são presos após roubarem veículo e carga dos Correios; mercadoria era transportada ...
São Luís, MA, BRAZIL           | VÍDEO: Motorista de ônibus é morto em assalto nesta segunda (22) em São Luís - Diário do Transp...
São Vicente, SP, BRAZIL        | Dois assaltos ocorrem no mesmo dia no bairro da Vila Margarida, em São Vicente - Boqnews


In [ ]:
stopwords_pt = set(STOPWORDS)
stopwords_pt.update([
    "de", "da", "do", "em", "na", "no", "com", "por", "para", "que", "um", "uma", "os", "as",
    "ao", "aos", "se", "foi", "pela", "pelo", "dos", "das", "brazil", "cidade", "estado",
    "região", "zona", "bairro", "município", "local", "dia", "noite", "hoje", "ontem"
])

top_cities = df_markov.sort_values(by='prob_assalto_futuro', ascending=False).head(4)['municipio'].tolist()

fig, axes = plt.subplots(2, 2, figsize=(16, 10))
axes = axes.flatten()

for i, cidade in enumerate(top_cities):
    textos_cidade = df_sample[df_sample["municipio"] == cidade]["descricao"].values

    texto_completo = " ".join(textos_cidade)

    wordcloud = WordCloud(
        width=800, height=400,
        background_color='white',
        stopwords=stopwords_pt,
        colormap='Reds',
        max_words=50,
        min_font_size=10
    ).generate(texto_completo)

    axes[i].imshow(wordcloud, interpolation="bilinear")

    nome_cidade = cidade.replace(", BRAZIL", "")
    axes[i].set_title(f"Padrão Semântico: {nome_cidade}", fontsize=14, fontweight='bold', pad=12)
    axes[i].axis('off')

plt.suptitle("Wordcloud: Termos mais frequentes nos Hubs de Criminalidade", fontsize=20, y=1)
plt.tight_layout()
plt.savefig("wordcloud_hubs_criminalidade.png", format="png", dpi=300, bbox_inches='tight')
plt.show()

A leitura das amostras confirma a robustez do modelo SBERT na formação do grafo. Observa-se que os municípios "atratores" (João Pessoa, São Carlos, etc.) compartilham gatilhos semânticos de alta gravidade:

* A predominância visual do termo "tiros" ao lado de "suspeito" e "homem" indica que estes clusters não agrupam apenas furtos simples, mas ocorrências de confronto direto e alta periculosidade. O modelo vetorial aproximou relatos onde a interação com o suspeito evolui para o disparo de arma de fogo.

* **Natureza do Delito**: A recorrência dos termos "assalto" e "roubo" confirma que o driver primário de conexão entre estas cidades é o crime patrimonial violento. O algoritmo detectou uma padronização na forma como estes delitos de abordagem direta são descritos pelas autoridades em diferentes estados.

* **Conclusão**: A alta probabilidade futura (Markov) destas cidades não é aleatória; ela reflete uma consistência sistêmica em crimes de roubo com emprego de violência, validando a capacidade do modelo de identificar "Zonas Quentes" de confronto armado.

**5. Diagnóstico de Conectividade e Ergodicidade**

Para assegurar a robustez do modelo preditivo, verifiquei a fragmentação da rede.

**Requisito Teórico**: Para que a Cadeia de Markov possua uma distribuição estacionária única e válida para todos os municípios, o grafo deve ser Conexo.

**Impacto**: Se houver múltiplos componentes isolados, um "caminhante aleatório" ficaria preso em um subconjunto de cidades, impedindo a análise global.



In [ ]:
# Análise de Componentes Conexos
num_components = nx.number_connected_components(G)
tamanho_componentes = [len(c) for c in nx.connected_components(G)]

print(f"Diagnóstico de Topologia:")
print(f"Número de Componentes: {num_components}")

if num_components == 1:
    print(f"CONCLUSÃO: O Grafo é Totalmente Conexo (Componente Gigante: {tamanho_componentes[0]} nós).")
    print("   - Implicação: A Cadeia de Markov é irredutível; o vetor estacionário é válido globalmente.")
else:
    print(f"ATENÇÃO: O grafo está fragmentado. Tamanhos: {tamanho_componentes}")
    print("   - Implicação: A análise deve ser feita separadamente para cada componente.")

Diagnóstico de Topologia:
Número de Componentes: 1
CONCLUSÃO: O Grafo é Totalmente Conexo (Componente Gigante: 318 nós).
   - Implicação: A Cadeia de Markov é irredutível; o vetor estacionário é válido globalmente.


**6. Teste de Calibração Algorítmica**

Para garantir que o método da potência não introduz viés numérico, submeti o algoritmo a um "Cenário de Máxima Entropia" (Matriz Uniforme).

**Hipótese**: Se todas as transições forem equiprováveis, o vetor estacionário deve convergir exata e uniformemente para $1/N$.

**Resultado Esperado**: A validação confirma que a implementação do algoritmo é neutra e preserva a distribuição de probabilidade.

In [ ]:
# Unit Test
uniform_matrix = np.ones((n, n)) / n
pi_test_uniform = np.ones(n) / n

# Executa iterações de teste
for _ in range(100):
    pi_next = pi_test_uniform @ uniform_matrix
    if np.allclose(pi_next, pi_test_uniform):
        break
    pi_test_uniform = pi_next

# Validação com Assert
expected_value = 1.0 / n
assert np.allclose(pi_test_uniform, expected_value), "Erro: O algoritmo falhou no teste uniforme."

print(f"Calibração Confirmada: Distribuição convergiu uniformemente para {expected_value:.6f}")

Calibração Confirmada: Distribuição convergiu uniformemente para 0.003145


**7. Quantificação do Erro de Simulação (MAE)**

A métrica final de validação é o Mean Absolute Error (MAE) entre o vetor teórico ($\pi$) e o vetor empírico obtido via Monte Carlo.

**Interpretação**: Um erro de magnitude desprezível ($< 10^{-3}$) confirma a Ergodicidade da cadeia, validando que a simulação numérica reproduz fielmente a topologia matemática do grafo.



In [ ]:
# Cálculo do Erro Médio Absoluto (MAE)
mae = np.abs(frequencia_simulada - pi).mean()

print(f"Relatório de Precisão da Simulação:")
print(f"Erro Médio Absoluto (MAE): {mae:.8f}")

if mae < 0.01:
    print("RESULTADO: A simulação é estatisticamente coerente com o modelo teórico.")
else:
    print("ATENÇÃO: Divergência significativa detectada. Verifique o número de passos da simulação.")

Relatório de Precisão da Simulação:
Erro Médio Absoluto (MAE): 0.00013669
RESULTADO: A simulação é estatisticamente coerente com o modelo teórico.


## Conclusão e Impacto do Modelo

Este estudo demonstrou a viabilidade de utilizar Processamento de Linguagem Natural (NLP) e Teoria dos Grafos para revelar estruturas latentes em dados de segurança pública, indo além das análises puramente geográficas.

1. **Validação Técnica**: A modelagem matemática provou-se robusta. A validação cruzada entre a matriz de transição teórica e a simulação de Monte Carlo (100.000 passos) apresentou uma convergência quase perfeita, com erro médio absoluto na ordem de $10^{-4}$. Isso confirma a integridade da Matriz de Transição Estocástica construída.

2. **Insights de Negócio**: O modelo identificou com sucesso "Hubs Semânticos" de criminalidade. Cidades geograficamente distantes (ex: João Pessoa e São Carlos) foram conectadas pela assinatura textual dos seus crimes. A inspeção qualitativa revelou que esses hubs concentram descrições de alta especificidade (violência armada, roubo a instituições financeiras), validando a capacidade do modelo de isolar padrões críticos automaticamente.

3. **Roadmap**: Como evolução deste trabalho, sugere-se a implementação de Grafos de Séries Temporais (Time-Series Graphs). Isso permitiria monitorar a evolução dinâmica desses clusters, possibilitando prever não apenas onde um tipo de crime pode ocorrer, mas quando um novo padrão narrativo começa a se disseminar pela rede.